In [1]:
import os
import glob
import pandas as pd
import csv
import zipfile
from rdkit.Chem import AllChem
pd.set_option('max_colwidth', 400)

Load the file with the structure you want to submit to Chem-GPS

In [2]:
use_cols = ['smiles_sanitized', 'inchi_sanitized']

df_DB = pd.read_csv('../outputs/tables/3_curated/curated_structure.tsv', 
                    sep ='\t', 
                    usecols = use_cols
                   )

df_DB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440224 entries, 0 to 440223
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   smiles_sanitized  440224 non-null  object
 1   inchi_sanitized   440224 non-null  object
dtypes: object(2)
memory usage: 6.7+ MB


Remove duplicated smiles

In [3]:
df_DB.drop_duplicates(['smiles_sanitized'],
                      inplace = True)
df_DB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428777 entries, 0 to 440223
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   smiles_sanitized  428777 non-null  object
 1   inchi_sanitized   428777 non-null  object
dtypes: object(2)
memory usage: 9.8+ MB


In [4]:
df_DB.reset_index(inplace = True)
df_DB.drop(['index'], axis = 1, inplace = True)
df_DB.reset_index(inplace = True)
df_DB['Cpd_id'] = df_DB['smiles_sanitized']+' '+df_DB['index'].astype(str)
df_DB

,index,smiles_sanitized,inchi_sanitized,Cpd_id
0,0,COc1cc2c(cc1O)C[C@H]1c3c(cc(O)c(OC)c3-2)CCN1C,"InChI=1S/C19H21NO4/c1-20-5-4-10-7-15(22)19(24-3)18-12-9-16(23-2)14(21)8-11(12)6-13(20)17(10)18/h7-9,13,21-22H,4-6H2,1-3H3/t13-/m0/s1",COc1cc2c(cc1O)C[C@H]1c3c(cc(O)c(OC)c3-2)CCN1C 0
1,1,COc1cc2c(cc1O)C[C@@H]1NCCc3cc4c(c-2c31)OCO4,"InChI=1S/C18H17NO4/c1-21-14-7-11-10(5-13(14)20)4-12-16-9(2-3-19-12)6-15-18(17(11)16)23-8-22-15/h5-7,12,19-20H,2-4,8H2,1H3/t12-/m0/s1",COc1cc2c(cc1O)C[C@@H]1NCCc3cc4c(c-2c31)OCO4 1
2,2,COc1cc2c(cc1OC)-c1c3c(cc4c1[C@H](C2)N(C)CC4)OCO3,"InChI=1S/C20H21NO4/c1-21-5-4-11-7-17-20(25-10-24-17)19-13-9-16(23-3)15(22-2)8-12(13)6-14(21)18(11)19/h7-9,14H,4-6,10H2,1-3H3/t14-/m0/s1",COc1cc2c(cc1OC)-c1c3c(cc4c1[C@H](C2)N(C)CC4)OCO3 2
3,3,COc1c2c3c(c4c1OCO4)-c1cc4c(cc1CC3NCC2)OCO4,"InChI=1S/C19H17NO5/c1-21-17-10-2-3-20-12-4-9-5-13-14(23-7-22-13)6-11(9)16(15(10)12)18-19(17)25-8-24-18/h5-6,12,20H,2-4,7-8H2,1H3",COc1c2c3c(c4c1OCO4)-c1cc4c(cc1CC3NCC2)OCO4 3
4,4,C[C@H]1[C@H](C)CC[C@]2(C)CC[C@]3(C)C(=CC[C@@H]4[C@@]5(C)CC[C@H](O)C(C)(C)[C@@H]5CC[C@]43C)[C@H]12,"InChI=1S/C30H50O/c1-19-11-14-27(5)17-18-29(7)21(25(27)20(19)2)9-10-23-28(6)15-13-24(31)26(3,4)22(28)12-16-30(23,29)8/h9,19-20,22-25,31H,10-18H2,1-8H3/t19-,20+,22+,23-,24+,25+,27-,28+,29-,30-/m1/s1",C[C@H]1[C@H](C)CC[C@]2(C)CC[C@]3(C)C(=CC[C@@H]4[C@@]5(C)CC[C@H](O)C(C)(C)[C@@H]5CC[C@]43C)[C@H]12 4
...,...,...,...,...
428772,428772,CC1(C)CC[C@]2(C=O)CCC3(C)C(=CCC4[C@@]5(C)CC[C@H](O)C(C)(C)C5CC[C@]43C)[C@@H]2C1,"InChI=1S/C30H48O2/c1-25(2)14-16-30(19-31)17-15-28(6)20(21(30)18-25)8-9-23-27(5)12-11-24(32)26(3,4)22(27)10-13-29(23,28)7/h8,19,21-24,32H,9-18H2,1-7H3/t21-,22?,23?,24-,27-,28?,29+,30+/m0/s1",CC1(C)CC[C@]2(C=O)CCC3(C)C(=CCC4[C@@]5(C)CC[C@H](O)C(C)(C)C5CC[C@]43C)[C@@H]2C1 428772
428773,428773,CC(C)[C@H](C)[C@@H](O)[C@H](O)[C@@H](C)[C@H]1CCC2[C@@H]3CC(=O)[C@H]4C[C@H](O)CC[C@]4(C)[C@H]3CC[C@@]21C,"InChI=1S/C28H48O4/c1-15(2)16(3)25(31)26(32)17(4)20-7-8-21-19-14-24(30)23-13-18(29)9-11-28(23,6)22(19)10-12-27(20,21)5/h15-23,25-26,29,31-32H,7-14H2,1-6H3/t16-,17-,18+,19-,20+,21?,22-,23+,25+,26+,27+,28+/m0/s1",CC(C)[C@H](C)[C@@H](O)[C@H](O)[C@@H](C)[C@H]1CCC2[C@@H]3CC(=O)[C@H]4C[C@H](O)CC[C@]4(C)[C@H]3CC[C@@]21C 428773
428774,428774,C[C@H]1CC[C@@H]2[C@@H](C)[C@H]3[C@H](C[C@H]4[C@@H]5CC=C6CC(O[C@@H]7O[C@H](CO)[C@@H](O)[C@H](O)[C@H]7O)CC[C@]6(C)C5CC[C@@]43C)N2C1,"InChI=1S/C33H53NO6/c1-17-5-8-24-18(2)27-25(34(24)15-17)14-23-21-7-6-19-13-20(9-11-32(19,3)22(21)10-12-33(23,27)4)39-31-30(38)29(37)28(36)26(16-35)40-31/h6,17-18,20-31,35-38H,5,7-16H2,1-4H3/t17-,18+,20?,21+,22?,23-,24+,25-,26+,27-,28+,29-,30+,31+,32-,33-/m0/s1",C[C@H]1CC[C@@H]2[C@@H](C)[C@H]3[C@H](C[C@H]4[C@@H]5CC=C6CC(O[C@@H]7O[C@H](CO)[C@@H](O)[C@H](O)[C@H]7O)CC[C@]6(C)C5CC[C@@]43C)N2C1 428774
428775,428775,C[C@H]1CC[C@@H]2[C@@H](C)[C@H]3[C@H](C[C@H]4[C@@H]5CC=C6CC(O[C@@H]7O[C@H](CO)[C@H](O)[C@H](O)[C@H]7O)CC[C@]6(C)C5CC[C@@]43C)N2C1,"InChI=1S/C33H53NO6/c1-17-5-8-24-18(2)27-25(34(24)15-17)14-23-21-7-6-19-13-20(9-11-32(19,3)22(21)10-12-33(23,27)4)39-31-30(38)29(37)28(36)26(16-35)40-31/h6,17-18,20-31,35-38H,5,7-16H2,1-4H3/t17-,18+,20?,21+,22?,23-,24+,25-,26+,27-,28-,29-,30+,31+,32-,33-/m0/s1",C[C@H]1CC[C@@H]2[C@@H](C)[C@H]3[C@H](C[C@H]4[C@@H]5CC=C6CC(O[C@@H]7O[C@H](CO)[C@H](O)[C@H](O)[C@H]7O)CC[C@]6(C)C5CC[C@@]43C)N2C1 428775


In [11]:
df_DB.loc[df_DB['smiles_sanitized'].str.contains('se',  case=True, regex=True) == True]

,index,smiles_sanitized,inchi_sanitized,Cpd_id,chunk
386151,386151,CCN1C(=CC=CC=Cc2[se]c3ccccc3[n+]2CC)[Se]c2ccccc21,"InChI=1S/C23H23N2Se2/c1-3-24-18-12-8-10-14-20(18)26-22(24)16-6-5-7-17-23-25(4-2)19-13-9-11-15-21(19)27-23/h5-17H,3-4H2,1-2H3/q+1",CCN1C(=CC=CC=Cc2[se]c3ccccc3[n+]2CC)[Se]c2ccccc21 386151,38
387890,387890,N=C(O)c1c[se]c(C2OC(CO)C(O)C2O)n1,"InChI=1S/C9H12N2O5Se/c10-8(15)3-2-17-9(11-3)7-6(14)5(13)4(1-12)16-7/h2,4-7,12-14H,1H2,(H2,10,15)",N=C(O)c1c[se]c(C2OC(CO)C(O)C2O)n1 387890,38
394911,394911,O=c1c2ccccc2[se]n1-c1ccccc1,InChI=1S/C13H9NOSe/c15-13-11-8-4-5-9-12(11)16-14(13)10-6-2-1-3-7-10/h1-9H,O=c1c2ccccc2[se]n1-c1ccccc1 394911,39
408222,408222,Cc1ccc(-c2nc(-c3ccccc3)c[se]2)cc1,"InChI=1S/C16H13NSe/c1-12-7-9-14(10-8-12)16-17-15(11-18-16)13-5-3-2-4-6-13/h2-11H,1H3",Cc1ccc(-c2nc(-c3ccccc3)c[se]2)cc1 408222,40


In [5]:
os.chdir("ChemGPS_NP_coordinates")

In [6]:
# Setting chunk size for upload on ChemGPS-NP
chunk_size = 10000

# Assigning chunk numbers to rows
df_DB['chunk'] = df_DB['index'].apply(lambda x: int(x / chunk_size))

# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df_DB.columns if col not in ['chunk', 'index', 'smiles_sanitized', 'inchi_sanitized']]

# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df_DB.groupby('chunk'):
    chunk[cols].to_csv(f'chunk{i}.txt', 
                       index = False,
                       header = False, 
                       quoting = csv.QUOTE_NONE,
                       quotechar = "",
                       escapechar = "\\"
                      )
    i += 1

Once this is done, submit each chunk file to ChemGPS-NP (https://chemgps.bmc.uu.se/batchelor/queue/submit?type=file) 

Once jobs are finished, download each zip file and put them all in "//ChemGPS-NP_inhouse_NPdb/Output_ChemGPS" folder

In [77]:
# This code is extracting each zip folder and keeps only the results files (chemgps.txt)

dir_name = 'C:/Users/gaudrya/Desktop/ChemGPS-NP_inhouse_NPdb/Output_ChemGPS'
extension = ".zip"
counter = 0

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file
        for filename in os.listdir(dir_name):
            if filename.endswith("chemgps.txt"):
                os.rename(os.path.join(dir_name,filename), os.path.join(dir_name,"chemgps"+str(counter)+'.txt'))
                counter = counter + 1
os.remove('README.txt') # delete README file 

In [78]:
os.chdir('C:/Users/gaudrya/Desktop/ChemGPS-NP_inhouse_NPdb/Output_ChemGPS/')

extension = 'txt'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f, sep='\t') for f in all_filenames ])

In [79]:
combined_csv

,MOLID,PS1,PS2,PS3,PS4,PS5,PS6,PS7,PS8
0,160000,2.669880,-2.284131,0.756952,-1.248140,-0.591557,0.751185,0.388124,-0.130381
1,160001,3.094192,-2.480459,0.140878,-1.278329,-0.644319,0.551546,0.424911,0.117496
2,160002,0.363029,-1.601661,1.732196,-0.783085,-0.065778,0.791331,0.500138,-0.629719
3,160003,1.543216,-1.841612,0.918328,-0.665443,-0.187714,0.680996,0.992040,-0.514296
4,160004,0.929476,-1.376261,0.712845,-1.004709,-0.298898,1.504837,1.297848,-0.718678
...,...,...,...,...,...,...,...,...,...
9995,99995,0.894463,-2.057577,1.140534,-0.984086,-0.491042,0.490950,-0.043388,-0.314483
9996,99996,-1.595748,3.595390,-0.081084,-0.472251,0.242734,-0.326062,-0.781273,0.267558
9997,99997,0.355959,2.834539,1.747885,-0.768920,0.271049,-0.153578,-1.178961,0.375431
9998,99998,-0.615692,-0.562537,1.480945,-0.434975,-0.005512,0.065329,-0.063173,-0.191786


In [80]:
df_DB

,index,sanitized_smiles,sanitized_inchi,Cpd_id,chunk
0,0,CC(C)=CCC/C(C)=C/COc1ccc(CCN(C)C(=O)/C=C/S(C)(=O)=O)cc1,"InChI=1S/C23H33NO4S/c1-19(2)7-6-8-20(3)14-17-28-22-11-9-21(10-12-22)13-16-24(4)23(25)15-18-29(5,26)27/h7,9-12,14-15,18H,6,8,13,16-17H2,1-5H3/b18-15+,20-14+",CC(C)=CCC/C(C)=C/COc1ccc(CCN(C)C(=O)/C=C/S(C)(=O)=O)cc1 0,0
1,1,C/C=C\C=C/CC/C=C/C=C/C(O)=NCC(C)C,"InChI=1S/C16H25NO/c1-4-5-6-7-8-9-10-11-12-13-16(18)17-14-15(2)3/h4-7,10-13,15H,8-9,14H2,1-3H3,(H,17,18)/b5-4-,7-6-,11-10+,13-12+",C/C=C\C=C/CC/C=C/C=C/C(O)=NCC(C)C 1,0
2,2,C/C=C/C=C\CC/C=C/C=C/C(O)=NCC(C)C,"InChI=1S/C16H25NO/c1-4-5-6-7-8-9-10-11-12-13-16(18)17-14-15(2)3/h4-7,10-13,15H,8-9,14H2,1-3H3,(H,17,18)/b5-4+,7-6-,11-10+,13-12+",C/C=C/C=C\CC/C=C/C=C/C(O)=NCC(C)C 2,0
3,3,OC(/C=C/c1ccccc1)=NCCc1ccccc1,"InChI=1S/C17H17NO/c19-17(12-11-15-7-3-1-4-8-15)18-14-13-16-9-5-2-6-10-16/h1-12H,13-14H2,(H,18,19)/b12-11+",OC(/C=C/c1ccccc1)=NCCc1ccccc1 3,0
4,4,C/C=C/C=C/CC/C=C/C=C/C(O)=NCC(C)C,"InChI=1S/C16H25NO/c1-4-5-6-7-8-9-10-11-12-13-16(18)17-14-15(2)3/h4-7,10-13,15H,8-9,14H2,1-3H3,(H,17,18)/b5-4+,7-6+,11-10+,13-12+",C/C=C/C=C/CC/C=C/C=C/C(O)=NCC(C)C 4,0
...,...,...,...,...,...
174953,174953,CON(CCCCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O,"InChI=1S/C28H46N2O6/c1-35-30(28-27(34)26(33)25(32)24(22-31)36-28)20-15-13-11-9-7-5-3-2-4-6-8-10-12-14-17-23-18-16-19-29-21-23/h16,18-19,21,24-28,31-34H,2-5,7,9-15,17,20,22H2,1H3",CON(CCCCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O 174953,17
174954,174954,CON(CCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O,"InChI=1S/C26H42N2O6/c1-33-28(26-25(32)24(31)23(30)22(20-29)34-26)18-13-11-9-7-5-3-2-4-6-8-10-12-15-21-16-14-17-27-19-21/h14,16-17,19,22-26,29-32H,2-3,5,7-13,15,18,20H2,1H3",CON(CCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O 174954,17
174955,174955,C1=CCCCCCCC2CN3CCC=CC=CCCCCc4cc(c[n+](c4)CCC=C1)C2CC3,"InChI=1S/C32H47N2/c1-2-5-10-14-18-23-34-26-29-19-15-11-7-4-6-9-13-17-22-33-24-21-32(31(25-29)28-34)30(27-33)20-16-12-8-3-1/h2,4-6,9-10,13-14,25-26,28,30,32H,1,3,7-8,11-12,15-24,27H2/q+1",C1=CCCCCCCC2CN3CCC=CC=CCCCCc4cc(c[n+](c4)CCC=C1)C2CC3 174955,17
174956,174956,OC1C=CC=CCCCCCCC2CN3CCC=CC=CCCCCC4=CC(CN(C4)C1)C2CC3,"InChI=1S/C32H50N2O/c35-31-19-15-11-7-2-1-6-10-14-18-29-25-33-21-16-12-8-4-3-5-9-13-17-28-23-30(32(29)20-22-33)26-34(24-28)27-31/h3-4,7-8,11-12,15,19,23,29-32,35H,1-2,5-6,9-10,13-14,16-18,20-22,24-27H2",OC1C=CC=CCCCCCCC2CN3CCC=CC=CCCCCC4=CC(CN(C4)C1)C2CC3 174956,17


In [89]:
df_merged = pd.merge(df_DB, combined_csv, left_on='index', right_on='MOLID', how='left')
df_merged

,index,sanitized_smiles,sanitized_inchi,Cpd_id,chunk,MOLID,PS1,PS2,PS3,PS4,PS5,PS6,PS7,PS8
0,0,CC(C)=CCC/C(C)=C/COc1ccc(CCN(C)C(=O)/C=C/S(C)(=O)=O)cc1,"InChI=1S/C23H33NO4S/c1-19(2)7-6-8-20(3)14-17-28-22-11-9-21(10-12-22)13-16-24(4)23(25)15-18-29(5,26)27/h7,9-12,14-15,18H,6,8,13,16-17H2,1-5H3/b18-15+,20-14+",CC(C)=CCC/C(C)=C/COc1ccc(CCN(C)C(=O)/C=C/S(C)(=O)=O)cc1 0,0,0,-0.176191,-0.076223,1.726788,1.103484,1.062404,-1.114490,0.725288,-0.298682
1,1,C/C=C\C=C/CC/C=C/C=C/C(O)=NCC(C)C,"InChI=1S/C16H25NO/c1-4-5-6-7-8-9-10-11-12-13-16(18)17-14-15(2)3/h4-7,10-13,15H,8-9,14H2,1-3H3,(H,17,18)/b5-4-,7-6-,11-10+,13-12+",C/C=C\C=C/CC/C=C/C=C/C(O)=NCC(C)C 1,0,1,-2.536835,-1.390764,0.990942,0.890511,1.130068,-1.342902,0.288192,-0.579178
2,2,C/C=C/C=C\CC/C=C/C=C/C(O)=NCC(C)C,"InChI=1S/C16H25NO/c1-4-5-6-7-8-9-10-11-12-13-16(18)17-14-15(2)3/h4-7,10-13,15H,8-9,14H2,1-3H3,(H,17,18)/b5-4+,7-6-,11-10+,13-12+",C/C=C/C=C\CC/C=C/C=C/C(O)=NCC(C)C 2,0,2,-2.536835,-1.390764,0.990942,0.890511,1.130068,-1.342902,0.288192,-0.579178
3,3,OC(/C=C/c1ccccc1)=NCCc1ccccc1,"InChI=1S/C17H17NO/c19-17(12-11-15-7-3-1-4-8-15)18-14-13-16-9-5-2-6-10-16/h1-12H,13-14H2,(H,18,19)/b12-11+",OC(/C=C/c1ccccc1)=NCCc1ccccc1 3,0,3,-2.002388,2.340355,0.790296,0.207338,1.051344,-0.874169,-0.443301,0.915898
4,4,C/C=C/C=C/CC/C=C/C=C/C(O)=NCC(C)C,"InChI=1S/C16H25NO/c1-4-5-6-7-8-9-10-11-12-13-16(18)17-14-15(2)3/h4-7,10-13,15H,8-9,14H2,1-3H3,(H,17,18)/b5-4+,7-6+,11-10+,13-12+",C/C=C/C=C/CC/C=C/C=C/C(O)=NCC(C)C 4,0,4,-2.536835,-1.390764,0.990942,0.890511,1.130068,-1.342902,0.288192,-0.579178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174953,174953,CON(CCCCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O,"InChI=1S/C28H46N2O6/c1-35-30(28-27(34)26(33)25(32)24(22-31)36-28)20-15-13-11-9-7-5-3-2-4-6-8-10-12-14-17-23-18-16-19-29-21-23/h16,18-19,21,24-28,31-34H,2-5,7,9-15,17,20,22H2,1H3",CON(CCCCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O 174953,17,174953,1.603456,-1.643537,0.918415,0.507250,0.210867,-1.891731,-1.222411,0.956592
174954,174954,CON(CCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O,"InChI=1S/C26H42N2O6/c1-33-28(26-25(32)24(31)23(30)22(20-29)34-26)18-13-11-9-7-5-3-2-4-6-8-10-12-15-21-16-14-17-27-19-21/h14,16-17,19,22-26,29-32H,2-3,5,7-13,15,18,20H2,1H3",CON(CCCCCCCCC#CCCCCc1cccnc1)C1OC(CO)C(O)C(O)C1O 174954,17,174954,1.194281,-1.465992,0.482938,0.401881,0.227754,-1.719457,-1.100658,0.978214
174955,174955,C1=CCCCCCCC2CN3CCC=CC=CCCCCc4cc(c[n+](c4)CCC=C1)C2CC3,"InChI=1S/C32H47N2/c1-2-5-10-14-18-23-34-26-29-19-15-11-7-4-6-9-13-17-22-33-24-21-32(31(25-29)28-34)30(27-33)20-16-12-8-3-1/h2,4-6,9-10,13-14,25-26,28,30,32H,1,3,7-8,11-12,15-24,27H2/q+1",C1=CCCCCCCC2CN3CCC=CC=CCCCCc4cc(c[n+](c4)CCC=C1)C2CC3 174955,17,174955,0.057956,-0.339736,3.323385,-1.107449,0.664955,1.584084,-0.619533,-0.541554
174956,174956,OC1C=CC=CCCCCCCC2CN3CCC=CC=CCCCCC4=CC(CN(C4)C1)C2CC3,"InChI=1S/C32H50N2O/c35-31-19-15-11-7-2-1-6-10-14-18-29-25-33-21-16-12-8-4-3-5-9-13-17-28-23-30(32(29)20-22-33)26-34(24-28)27-31/h3-4,7-8,11-12,15,19,23,29-32,35H,1-2,5-6,9-10,13-14,16-18,20-22,24-27H2",OC1C=CC=CCCCCCCC2CN3CCC=CC=CCCCCC4=CC(CN(C4)C1)C2CC3 174956,17,174956,0.314736,-1.652390,2.757961,-1.185565,0.411471,1.723506,-0.374881,-0.821542


In [92]:
df_merged_clean = df_merged.drop(['index', 'Cpd_id', 'chunk', 'MOLID'], axis=1)


df_merged_clean.to_csv('C:/Users/gaudrya/Desktop/ChemGPS-NP_inhouse_NPdb/open_NP_db_sanitized_with_coordinates.tsv',
                       sep='\t')